# Conexión a la base de datos

In [10]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

load_dotenv()

pg_host = os.getenv("PG_HOST")
pg_db   = os.getenv("PG_DB")
pg_user = os.getenv("PG_USER")
pg_pass = os.getenv("PG_PASS")
puerto = os.getenv("PG_PORT", "5432") 

# 3. Crear la cadena de conexión
cadena_conexion = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{puerto}/{pg_db}'

# 4. Crear el motor de SQLAlchemy
engine = create_engine(cadena_conexion)

# 5. Cargar los datos en un DataFrame de Pandas
try:
    consulta_sql = "SELECT * FROM mediciones;"
    df = pd.read_sql_query(consulta_sql, engine)
    print("Conexión exitosa y DataFrame creado.")
    print(df.head())
except Exception as ex:
    print("Error al conectar a la base de datos o cargar los datos:", ex)


Conexión exitosa y DataFrame creado.
   id       fecha    ph  conductividad  turbidez potable
0   1  2015-07-07  6.88         1479.0     890.0       N
1   2  2019-12-09  7.60         1653.0    1207.0       N
2   3  2020-01-30  6.69         1049.0    1667.0       N
3   4  2020-09-25  6.56         1651.0    1150.0       N
4   5  2021-01-12  7.02         1696.0    1050.0       N


# EDA

## Forma del dataset

In [11]:
print("Tamaño del DataFrame:", df.shape)

Tamaño del DataFrame: (213, 6)


## Primeras/últimas filas

In [16]:
df.head()

,id,fecha,ph,conductividad,turbidez,potable
0,1,2015-07-07,6.88,1479.0,890.0,N
1,2,2019-12-09,7.60,1653.0,1207.0,N
2,3,2020-01-30,6.69,1049.0,1667.0,N
3,4,2020-09-25,6.56,1651.0,1150.0,N
4,5,2021-01-12,7.02,1696.0,1050.0,N


In [17]:
df.tail()

,id,fecha,ph,conductividad,turbidez,potable
208,209,2022-09-16,6.6,1520.0,1300.0,N
209,210,2022-09-17,7.7,1740.0,1250.0,N
210,211,2022-09-18,6.5,1620.0,1400.0,N
211,212,2022-09-19,7.8,1800.0,950.0,N
212,213,2022-09-20,6.4,1450.0,1500.0,N


## Tipos de datos

In [18]:
df.dtypes

id                 int64
fecha             object
ph               float64
conductividad    float64
turbidez         float64
potable           object
dtype: object

## Información resumida

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             213 non-null    int64  
 1   fecha          213 non-null    object 
 2   ph             213 non-null    float64
 3   conductividad  213 non-null    float64
 4   turbidez       213 non-null    float64
 5   potable        213 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 10.1+ KB


Se cambiará el tipo de variable de fecha a tipo date por si es necesario realizar operaciones temporales y la varieble potable a tipo factor

In [19]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['potable'] = df['potable'].astype('category')

In [21]:
codigos, categorias = df["potable"].factorize()
df["potable_codificada"] = codigos

print("Códigos asignados:")
print(pd.DataFrame({"Código": range(len(categorias)), "Categoría": categorias}))

Códigos asignados:
   Código Categoría
0       0         N
1       1         S


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  213 non-null    int64         
 1   fecha               213 non-null    datetime64[ns]
 2   ph                  213 non-null    float64       
 3   conductividad       213 non-null    float64       
 4   turbidez            213 non-null    float64       
 5   potable             213 non-null    category      
 6   potable_codificada  213 non-null    int64         
dtypes: category(1), datetime64[ns](1), float64(3), int64(2)
memory usage: 10.4 KB


In [23]:
df.head()  # Muestra las primeras filas del DataFrame

,id,fecha,ph,conductividad,turbidez,potable,potable_codificada
0,1,2015-07-07,6.88,1479.0,890.0,N,0
1,2,2019-12-09,7.60,1653.0,1207.0,N,0
2,3,2020-01-30,6.69,1049.0,1667.0,N,0
3,4,2020-09-25,6.56,1651.0,1150.0,N,0
4,5,2021-01-12,7.02,1696.0,1050.0,N,0


## Tratamiento de valores nulos

In [14]:
df.isnull().sum()

id               0
fecha            0
ph               0
conductividad    0
turbidez         0
potable          0
dtype: int64

## Estadísticas Descriptivas

### Funciones

In [36]:
def analisis_descriptivo(datos, nombre_variable):
    """
    Realiza un análisis descriptivo completo de una variable numérica.

    Args:
        datos (pd.Series): Serie de pandas con los datos a analizar
        nombre_variable (str): Nombre de la variable para los resultados

    Returns:
        dict: Diccionario con todas las medidas estadísticas
    """
    if isinstance(datos, pd.Series):
        datos_series = datos
    else:
        datos_series = pd.Series(datos)

    # Cálculos básicos
    modas = datos_series.mode()
    moda = modas[0] if not modas.empty else None

    q1 = datos_series.quantile(0.25)
    q3 = datos_series.quantile(0.75)
    iqr = q3 - q1

    # Detección de outliers
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    outliers = datos_series[(datos_series < limite_inferior) | (datos_series > limite_superior)]

    # Resultados numéricos
    resultados = {
        'Variable': nombre_variable,
        'N': int(len(datos_series)),
        'Media': float(datos_series.mean()),
        'Mediana': float(datos_series.median()),
        'Moda': float(moda) if moda is not None else None,
        'Mínimo': int(datos_series.min()),
        'Máximo': float(datos_series.max()),
        'Rango': float(datos_series.max() - datos_series.min()),
        'Varianza': float(datos_series.var(ddof=0)),
        'Desviación Estándar': float(datos_series.std(ddof=0)),
        'Coeficiente de Variación': float((datos_series.std(ddof=0) / datos_series.mean()) * 100) if datos_series.mean() != 0 else float('nan'),
        'Q1': float(q1),
        'Q3': float(q3),
        'Rango Intercuartílico': int(iqr),
        'Asimetría': float(datos_series.skew()),
        'Curtosis': float(datos_series.kurtosis() + 3),
        'Número de Outliers': int(len(outliers)),
        'Porcentaje Outliers': float(len(outliers)/len(datos_series)*100)
    }

    # Interpretación automática
    skew = resultados['Asimetría']
    kurt = resultados['Curtosis']
    cv = resultados['Coeficiente de Variación']

    interpretacion = {
        'Asimetría': 'Asimetría positiva fuerte (cola derecha)' if skew > 1 else
                     'Asimetría positiva moderada' if skew > 0.5 else
                     'Simétrica' if -0.5 <= skew <= 0.5 else
                     'Asimetría negativa moderada' if skew < -0.5 else
                     'Asimetría negativa fuerte (cola izquierda)',
        'Curtosis': 'Distribución muy leptocúrtica (puntiaguda con colas pesadas)' if kurt > 4 else
                    'Distribución leptocúrtica' if kurt > 3 else
                    'Distribución mesocúrtica (normal)' if 2.5 <= kurt <= 3.5 else
                    'Distribución platicúrtica (plana)',
        'Variabilidad': 'Variabilidad extremadamente alta' if cv > 100 else
                        'Variabilidad muy alta' if cv > 50 else
                        'Variabilidad moderada' if cv > 20 else
                        'Variabilidad baja'
    }

    resultados['Interpretación'] = interpretacion
    return resultados

### Analisis

In [38]:
analisis_descriptivo_ph = analisis_descriptivo(df['ph'], 'ph')
print("Análisis descriptivo del pH:")
analisis_descriptivo_ph


Análisis descriptivo del pH:


{'Variable': 'ph',
 'N': 213,
 'Media': 7.027558685446009,
 'Mediana': 7.0,
 'Moda': 6.5,
 'Mínimo': 6,
 'Máximo': 7.9,
 'Rango': 1.6000000000000005,
 'Varianza': 0.22150014326963344,
 'Desviación Estándar': 0.4706380172379123,
 'Coeficiente de Variación': 6.697034323065819,
 'Q1': 6.6,
 'Q3': 7.45,
 'Rango Intercuartílico': 0,
 'Asimetría': 0.21055209293696378,
 'Curtosis': 1.8601020157523205,
 'Número de Outliers': 0,
 'Porcentaje Outliers': 0.0,
 'Interpretación': {'Asimetría': 'Simétrica',
  'Curtosis': 'Distribución platicúrtica (plana)',
  'Variabilidad': 'Variabilidad baja'}}

In [39]:
analisis_descriptivo_conductividad = analisis_descriptivo(df['conductividad'], 'conductividad')
print("Análisis descriptivo de la conductividad:")
analisis_descriptivo_conductividad

Análisis descriptivo de la conductividad:


{'Variable': 'conductividad',
 'N': 213,
 'Media': 1623.5727699530516,
 'Mediana': 1655.0,
 'Moda': 1675.0,
 'Mínimo': 1049,
 'Máximo': 1825.0,
 'Rango': 776.0,
 'Varianza': 16844.197756177124,
 'Desviación Estándar': 129.78519852501333,
 'Coeficiente de Variación': 7.993802367649113,
 'Q1': 1585.0,
 'Q3': 1690.0,
 'Rango Intercuartílico': 105,
 'Asimetría': -1.6047629014424771,
 'Curtosis': 7.014203472661007,
 'Número de Outliers': 23,
 'Porcentaje Outliers': 10.7981220657277,
 'Interpretación': {'Asimetría': 'Asimetría negativa moderada',
  'Curtosis': 'Distribución muy leptocúrtica (puntiaguda con colas pesadas)',
  'Variabilidad': 'Variabilidad baja'}}

In [40]:
analisis_descriptivo_turbidez = analisis_descriptivo(df['turbidez'], 'turbidez')
print("Análisis descriptivo de la turbidez:")   
analisis_descriptivo_turbidez

Análisis descriptivo de la turbidez:


{'Variable': 'turbidez',
 'N': 213,
 'Media': 1173.605633802817,
 'Mediana': 1100.0,
 'Moda': 1050.0,
 'Mínimo': 880,
 'Máximo': 1670.0,
 'Rango': 790.0,
 'Varianza': 38308.623818025524,
 'Desviación Estándar': 195.72588949350958,
 'Coeficiente de Variación': 16.677313388425198,
 'Q1': 1035.0,
 'Q3': 1300.0,
 'Rango Intercuartílico': 265,
 'Asimetría': 0.9005806325147123,
 'Curtosis': 2.843972627555225,
 'Número de Outliers': 0,
 'Porcentaje Outliers': 0.0,
 'Interpretación': {'Asimetría': 'Asimetría positiva moderada',
  'Curtosis': 'Distribución mesocúrtica (normal)',
  'Variabilidad': 'Variabilidad baja'}}